In [3]:
from tqdm import tqdm
import h5py
import pickle
import os 
os.chdir('/home/shimingwang/workspace/sf_tv/sceneflow_tv')

In [4]:
data_root = 'data/waymo_open_dataset_scene_flow/preprocess_v2/train'
file_name = 'index_total.pkl'
file_path = os.path.join(data_root, file_name)

total_index = pickle.load(open(file_path, 'rb'))

In [5]:
no_lable_list = []
for idx in tqdm(range(len(total_index))):
    scene_id, timestamp = total_index[idx]
    key = str(timestamp)
    with h5py.File(os.path.join(data_root, f'{scene_id}.h5'), 'r') as f:
        # print(f[key].keys())
        if 'label' not in f[key].keys():
            no_lable_list.append([scene_id, timestamp])
print(no_lable_list)

100%|██████████| 155687/155687 [00:19<00:00, 8086.11it/s]

[['2088865281951278665_4460_000_4480_000', '1553535058789636'], ['2088865281951278665_4460_000_4480_000', '1553535058889616'], ['2088865281951278665_4460_000_4480_000', '1553535059489540'], ['2088865281951278665_4460_000_4480_000', '1553535061389604'], ['2088865281951278665_4460_000_4480_000', '1553535061489589'], ['2088865281951278665_4460_000_4480_000', '1553535062189579'], ['2088865281951278665_4460_000_4480_000', '1553535062289602'], ['2088865281951278665_4460_000_4480_000', '1553535062389615'], ['2088865281951278665_4460_000_4480_000', '1553535062489637'], ['2088865281951278665_4460_000_4480_000', '1553535062589656'], ['2088865281951278665_4460_000_4480_000', '1553535062689646'], ['2088865281951278665_4460_000_4480_000', '1553535062789669'], ['2088865281951278665_4460_000_4480_000', '1553535062889647'], ['2088865281951278665_4460_000_4480_000', '1553535062989672'], ['2088865281951278665_4460_000_4480_000', '1553535063089655'], ['2088865281951278665_4460_000_4480_000', '15535350631

In [6]:
print(len(no_lable_list))
for no_label_sample in no_lable_list:
    scene_id, timestamp = no_label_sample
    # print(scene_id, timestamp)

33


In [7]:
from src.dataset import HDF5Dataset, collate_fn_pad
from torch.utils.data import DataLoader
from tqdm import tqdm

In [13]:
train_dataset = HDF5Dataset(data_root, n_frames=2, dufo=True)
train_data = DataLoader(train_dataset,
                        batch_size=1,
                        shuffle=True,
                        num_workers=8,
                        collate_fn=collate_fn_pad,
                        pin_memory=True)

In [14]:
for idx, data in tqdm(enumerate(train_data)):
    pass

6407it [00:26, 240.52it/s]


KeyError: Caught KeyError in DataLoader worker process 7.
Original Traceback (most recent call last):
  File "/home/shimingwang/miniconda3/envs/sf_tv/lib/python3.8/site-packages/torch/utils/data/_utils/worker.py", line 308, in _worker_loop
    data = fetcher.fetch(index)
  File "/home/shimingwang/miniconda3/envs/sf_tv/lib/python3.8/site-packages/torch/utils/data/_utils/fetch.py", line 51, in fetch
    data = [self.dataset[idx] for idx in possibly_batched_index]
  File "/home/shimingwang/miniconda3/envs/sf_tv/lib/python3.8/site-packages/torch/utils/data/_utils/fetch.py", line 51, in <listcomp>
    data = [self.dataset[idx] for idx in possibly_batched_index]
  File "/home/shimingwang/workspace/sf_tv/sceneflow_tv/src/dataset.py", line 197, in __getitem__
    res_dict['pc0_dynamic'] = torch.tensor(f[key]['label'][:].astype('int16'))
  File "h5py/_objects.pyx", line 54, in h5py._objects.with_phil.wrapper
  File "h5py/_objects.pyx", line 55, in h5py._objects.with_phil.wrapper
  File "/home/s

## Check waymo validation data

In [2]:
from tqdm import tqdm
import h5py
import pickle
import os 
os.chdir('/home/shimingwang/workspace/sf_tv/sceneflow_tv')

from src.trainer import ModelWrapper
import hydra
from hydra.core.hydra_config import HydraConfig
import omegaconf 
from omegaconf import OmegaConf, DictConfig, ListConfig
from src.dataset import HDF5Dataset, collate_fn_pad
from torch.utils.data import DataLoader
from tqdm import tqdm
import torch

In [31]:
val_dataset = HDF5Dataset('data/waymo_open_dataset_scene_flow/preprocess/valid', #"/val", 
                n_frames=2,
                )
val_loader = DataLoader(
    val_dataset,
    batch_size=1,
    shuffle=False,
collate_fn=collate_fn_pad,
pin_memory=True)

In [4]:
checkpoint_path = 'logs/jobs/sf_voxel_model_m_16_n_128_ep_12_lr_0.0002_bs_10_use_gru_decoder_m_16_n_128_4_a100_new_fine_tune_ep6/11-09-15-01/checkpoints/last.ckpt'
config_path = 'conf/eval.yaml'

In [5]:
cfg = OmegaConf.load(config_path)
cfg.update(checkpoint=checkpoint_path)
print(cfg.checkpoint)
torch_load_ckpt = torch.load(cfg.checkpoint)
checkpoint_params = DictConfig(torch_load_ckpt["hyper_parameters"])
print(checkpoint_params.cfg.model)
# print(cfg.defaults.model)
cfg.update(model=checkpoint_params.cfg.model)

logs/jobs/sf_voxel_model_m_16_n_128_ep_12_lr_0.0002_bs_10_use_gru_decoder_m_16_n_128_4_a100_new_fine_tune_ep6/11-09-15-01/checkpoints/last.ckpt
{'name': 'sf_voxel_model', 'target': {'_target_': 'src.models.SFVoxelModel', 'nframes': 1, 'm': 16, 'n': 128, 'input_channels': 32, 'output_channels': 64, 'point_cloud_range': [-51.2, -51.2, -3, 51.2, 51.2, 3], 'voxel_size': [0.2, 0.2, 6], 'grid_feature_size': [512, 512, 1], 'decoder': 'gru_decoder', 'decoder_layers': 1, 'use_bn_in_vol': True, 'use_ball_query': False, 'vol_conv_hidden_dim': 16}, 'val_monitor': 'val/Dynamic/Mean', 'exp_id': 'sf_voxel_model_m_16_n_128'}


In [7]:
mymodel = ModelWrapper.load_from_checkpoint(cfg.checkpoint, cfg=cfg, eval=True)

In wrapper:  <class 'omegaconf.dictconfig.DictConfig'>
n x/y/z:  22 22 2
search window radius in target pc: 11
using decoder: gru_decoder
using knn, m=16, n=128
gru decoder: GRUDecoder(
  (offset_encoder): Linear(in_features=3, out_features=64, bias=True)
  (proj_head): Sequential(
    (0): Linear(in_features=192, out_features=128, bias=True)
    (1): ReLU()
  )
  (gru): ConvGRU(
    (convz): Linear(in_features=192, out_features=128, bias=True)
    (convr): Linear(in_features=192, out_features=128, bias=True)
    (convq): Linear(in_features=192, out_features=128, bias=True)
    (sigmoid): Sigmoid()
    (tanh): Tanh()
  )
  (decoder): Sequential(
    (0): Linear(in_features=192, out_features=32, bias=True)
    (1): GELU(approximate='none')
    (2): Linear(in_features=32, out_features=3, bias=True)
  )
)


In [26]:
def batch_cuda(batch):
    for key in batch.keys():
        if isinstance(batch[key], torch.Tensor):
            batch[key] = batch[key].cuda()
        elif isinstance(batch[key], list):
            batch[key][0] = batch[key][0].cuda()
        else:
            raise ValueError
    return batch    

In [38]:
index = 29457
error_list = []
mymodel.model = mymodel.model.cuda()
for idx, batch in tqdm(enumerate(val_loader)):
    if idx >= index:
        batch = batch_cuda(batch)
        try:
            res_dict = mymodel.model(batch)
        except:
            error_list.append((idx, batch))
            print('Error catched for idx:', idx)

29462it [05:28, 86.09it/s] 

Error catched for idx: 29457
error in im2ht_gpu_kernel: invalid configuration argument


39381it [14:57, 43.90it/s]


In [37]:
print(batch.keys())
for key in batch.keys():
    if isinstance(batch[key], torch.Tensor):
        print(key, batch[key].shape)
    elif isinstance(batch[key], list):
        print(key, batch[key][0].shape)
    else:
        raise ValueError

dict_keys(['pc0', 'pc1', 'pose0', 'pose1', 'flow', 'flow_is_valid', 'flow_category_indices', 'ego_motion'])
pc0 torch.Size([1, 1248, 3])
pc1 torch.Size([1, 895, 3])
pose0 torch.Size([4, 4])
pose1 torch.Size([4, 4])
flow torch.Size([1, 1248, 3])
flow_is_valid torch.Size([1, 1248])
flow_category_indices torch.Size([1, 1248])
ego_motion torch.Size([4, 4])
